In [1]:
import pandas as pd
import numpy as np
import psycopg2
import math
from datetime import timedelta

%matplotlib inline

con = psycopg2.connect (user = 'postgres',
                password = 'postgres',
                port = '5433',
                host = 'localhost',
                database = 'inicudb')

cur  = con.cursor()

In [2]:
uhidPatient = "RNEH.0000013713"
#uhidPatient = "RSHI.0000012984"
caseType = 'Death'
#caseType = 'Discharge'
seperator = '_'

In [3]:
data = pd.read_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_1.csv')

In [4]:
data['series'] = data.hour_series

In [5]:
data.columns

Index(['Unnamed: 0', 'actual_DOB', 'uhid', 'dischargeddate', 'actual_DOA',
       'gender', 'birthweight', 'birthlength', 'birthheadcircumference',
       'inout_patient_status', 'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'dischargestatus', 'birthlength.1',
       'birthheadcircumference.1', 'inout_patient_status.1',
       'gestationweekbylmp.1', 'gestationdaysbylmp.1', 'baby_type.1',
       'central_temp.1', 'apgar_onemin.1', 'apgar_fivemin.1', 'apgar_tenmin.1',
       'motherage.1', 'conception_type.1', 'mode_of_delivery.1',
       'steroidname.1', 'numberofdose.1', 'los', 'day_1', 'hour_series',
       'hour', 'day_x', 'abdomen_girth', 'urine', 'stool', 'stool_passed',
       'entry_timestamp', 'day_y', 'time_divide', 'series'],
      dtype='object')

In [6]:
data.urine


0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
1520    25.0
1521    25.0
1522    25.0
1523    25.0
1524    25.0
Name: urine, Length: 1525, dtype: float64

In [7]:
data.time_divide.unique()

array([    0.   , 73787.556, 14376.503])

In [8]:
def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

def split_hour(x):
    try:
        t=str(x)
        return t.split(":")[0]
    except:
        pass
    
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass
    

In [9]:
def urine_check(x):
    try:
        if x > 200:
            return x/10
        else:
            return x
    except:
        pass

In [10]:
def to_float(x):
    if x == '':
        return np.nan
    else:
        return float(x)

In [11]:
dg = data.copy()

In [12]:
dg['urine'] = dg['urine'].apply(to_float)

In [13]:
dg['urine'] = dg['urine'].apply(urine_check)

In [14]:
dg['urine_per_hour'] = dg['urine']/((dg['time_divide'])/(3600))

In [15]:
dt = dg.sort_values(by = ['uhid','hour_series'])

In [16]:
ds = pd.DataFrame()

In [17]:
for i in dt.uhid.unique():
    x = dt[dt['uhid']==i]
    x['urine_per_hour'].fillna(method='bfill',inplace=True)
    ds = ds.append(x,ignore_index=True)
       

In [18]:
cur1 = con.cursor()
cur1.execute("SELECT uhid,creationtime,modificationtime,ph,entrydate FROM apollo.nursing_bloodgas where uhid ='"+ uhidPatient+"'")
cols1 = list(map(lambda x: x[0],cur1.description))
ph = pd.DataFrame(cur1.fetchall(),columns=cols1)

In [19]:
ph

,uhid,creationtime,modificationtime,ph,entrydate
0,RNEH.0000013713,2020-01-08 06:00:49.956,2020-01-08 06:00:49.956,7.012,2020-01-08 11:30:49.956


In [20]:
def ph_func(x):
    try:
        p = float(x)
        if p>=6.80 and p<=7.8:
            return p
        else:
            return None
    except:
        pass

In [21]:
ph

,uhid,creationtime,modificationtime,ph,entrydate
0,RNEH.0000013713,2020-01-08 06:00:49.956,2020-01-08 06:00:49.956,7.012,2020-01-08 11:30:49.956


In [22]:
ph['new_ph'] = ph.ph.apply(ph_func)

In [23]:
ph.dropna(subset=['new_ph'],inplace=True)

In [24]:
ph_df = pd.DataFrame()
for i in ph.uhid.unique():
    x = ph[ph['uhid']==i]
    ph_df = ph_df.append(x,ignore_index=True)

In [25]:
ph

,uhid,creationtime,modificationtime,ph,entrydate,new_ph
0,RNEH.0000013713,2020-01-08 06:00:49.956,2020-01-08 06:00:49.956,7.012,2020-01-08 11:30:49.956,7.012


In [26]:
ph_df

,uhid,creationtime,modificationtime,ph,entrydate,new_ph
0,RNEH.0000013713,2020-01-08 06:00:49.956,2020-01-08 06:00:49.956,7.012,2020-01-08 11:30:49.956,7.012


In [27]:
#ph_df = pd.DataFrame(columns = ph.columns)


In [28]:
ph_df

,uhid,creationtime,modificationtime,ph,entrydate,new_ph
0,RNEH.0000013713,2020-01-08 06:00:49.956,2020-01-08 06:00:49.956,7.012,2020-01-08 11:30:49.956,7.012


In [29]:
ph_df_1 = None

In [30]:
if len(ph_df.columns)>0:
    ph_df['hour'] = ph_df.entrydate.apply(split_hour)
    ph_df.drop_duplicates(subset=['uhid','hour'],keep='last',inplace=True)
else:
    x = pd.DataFrame(data=[uhidPatient],columns={'uhid'})
    print(x)
    ph_df_1 = pd.DataFrame(columns=ph.columns)
    ph_df_1 = ph_df_1.append(x,ignore_index=True)

In [31]:
if not(ph_df_1 is None) and len(ph_df_1.columns)>0 :
    ph_df_1

In [32]:
type(ds.uhid[0])

str

In [33]:
ds['uhid'] = ds['uhid'].astype(str)


In [34]:
if len(ph_df.columns)>0:
    ss = pd.merge(ds,ph_df,on=['uhid','hour'],how='left')
    ss = ss.drop_duplicates(subset=['uhid','hour_series'],keep='first')
else:
    ss = pd.merge(ds,ph_df_1,on=['uhid'],how='left')
    ss = ss.drop_duplicates(subset=['uhid','hour_series'],keep='first')

In [35]:
ss

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,entry_timestamp,day_y,time_divide,series,urine_per_hour,creationtime,modificationtime,ph,entrydate,new_ph
0,0,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:31:00,1.219718,NaT,NaT,NaN,NaT,NaN
1,1,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:32:00,1.219718,NaT,NaT,NaN,NaT,NaN
2,2,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:33:00,1.219718,NaT,NaT,NaN,NaT,NaN
3,3,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:34:00,1.219718,NaT,NaT,NaN,NaT,NaN
4,4,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:35:00,1.219718,NaT,NaT,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,1520,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08 14:00:24.059000+08:00,2020-01-08,0.0,2020-01-08 14:51:00,inf,NaT,NaT,NaN,NaT,NaN
1521,1521,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08 14:00:24.059000+08:00,2020-01-08,0.0,2020-01-08 14:52:00,inf,NaT,NaT,NaN,NaT,NaN
1522,1522,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08 14:00:24.059000+08:00,2020-01-08,0.0,2020-01-08 14:53:00,inf,NaT,NaT,NaN,NaT,NaN
1523,1523,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08 14:00:24.059000+08:00,2020-01-08,0.0,2020-01-08 14:54:00,inf,NaT,NaT,NaN,NaT,NaN


In [36]:
ss

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,entry_timestamp,day_y,time_divide,series,urine_per_hour,creationtime,modificationtime,ph,entrydate,new_ph
0,0,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:31:00,1.219718,NaT,NaT,NaN,NaT,NaN
1,1,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:32:00,1.219718,NaT,NaT,NaN,NaT,NaN
2,2,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:33:00,1.219718,NaT,NaT,NaN,NaT,NaN
3,3,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:34:00,1.219718,NaT,NaT,NaN,NaT,NaN
4,4,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,NaN,0.0,2020-01-07 13:35:00,1.219718,NaT,NaT,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,1520,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08 14:00:24.059000+08:00,2020-01-08,0.0,2020-01-08 14:51:00,inf,NaT,NaT,NaN,NaT,NaN
1521,1521,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08 14:00:24.059000+08:00,2020-01-08,0.0,2020-01-08 14:52:00,inf,NaT,NaT,NaN,NaT,NaN
1522,1522,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08 14:00:24.059000+08:00,2020-01-08,0.0,2020-01-08 14:53:00,inf,NaT,NaT,NaN,NaT,NaN
1523,1523,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08 14:00:24.059000+08:00,2020-01-08,0.0,2020-01-08 14:54:00,inf,NaT,NaT,NaN,NaT,NaN


In [37]:
len(ss.uhid.unique())

1

In [38]:
cur3 = con.cursor()
cur3.execute("SELECT t1.uhid,t1.entrydate,t1.rbs,t1.skintemp,t1.centraltemp  FROM apollo.nursing_vitalparameters AS t1 where t1.uhid = '"+uhidPatient+"'")
cols3 = list(map(lambda x: x[0], cur3.description))
nv = pd.DataFrame(cur3.fetchall(),columns=cols3)

In [39]:
nv.drop_duplicates(subset=['uhid','entrydate'],keep='first',inplace=True)

In [40]:
nv['hour'] = nv.entrydate.apply(split_hour)

In [41]:
nv.hour.iloc[0]

'2020-01-07 19'

In [42]:
ss.uhid = ss.uhid.astype(str)

In [43]:
ss.hour.iloc[0]

'2020-01-07 13'

In [44]:
s1 = pd.merge(ss,nv,on = ['uhid','hour'], how='left')

In [45]:
len(s1.uhid.unique())

1

In [46]:
s1.drop_duplicates(subset=['uhid','hour_series'],keep='first',inplace=True)

In [47]:
s1

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,urine_per_hour,creationtime,modificationtime,ph,entrydate_x,new_ph,entrydate_y,rbs,skintemp,centraltemp
0,0,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,1.219718,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
1,1,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,1.219718,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
2,2,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,1.219718,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
3,3,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,1.219718,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
4,4,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,1.219718,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,1520,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,inf,NaT,NaT,NaN,NaT,NaN,2020-01-08 14:00:11.064,NaN,None,36.2
1521,1521,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,inf,NaT,NaT,NaN,NaT,NaN,2020-01-08 14:00:11.064,NaN,None,36.2
1522,1522,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,inf,NaT,NaT,NaN,NaT,NaN,2020-01-08 14:00:11.064,NaN,None,36.2
1523,1523,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,inf,NaT,NaT,NaN,NaT,NaN,2020-01-08 14:00:11.064,NaN,None,36.2


In [48]:
def temp_a(x,y):
    if not(x is None) and (x>0):
        return x
    else:
        return y

In [49]:
s1['temp'] = s1.apply(lambda x: temp_a(x['centraltemp'], x['skintemp']), axis=1)

In [50]:
cur2 = con.cursor()
cur2.execute("SELECT t1.uhid,t1.visitdate,t1.visittime,t1.currentdateweight, t1.currentdateheight  FROM apollo.baby_visit AS t1 where t1.uhid = '"+uhidPatient+"'")
cols2 = list(map(lambda x: x[0], cur2.description))
bv = pd.DataFrame(cur2.fetchall(),columns=cols2)

In [51]:
bv

,uhid,visitdate,visittime,currentdateweight,currentdateheight
0,RNEH.0000013713,2020-01-07,12:29:00,2846.0,None


In [52]:
bv.sort_values('visitdate',inplace=True)

In [53]:
def stamp_1(x,y):
    return str(x) +" "+ str(y).split(":")[0]

In [54]:
bv

,uhid,visitdate,visittime,currentdateweight,currentdateheight
0,RNEH.0000013713,2020-01-07,12:29:00,2846.0,None


In [55]:
bv['hour'] = bv.apply(lambda x: stamp_1(x['visitdate'], x['visittime']), axis=1)

In [56]:
def to_str(x):
    return (str(x))

In [57]:
bv['day'] = bv['visitdate'].apply(to_str)

In [58]:
bv.drop_duplicates(subset=['uhid','visitdate'],keep='last',inplace=True)

In [59]:
s1['day'] = s1['hour_series'].apply(split_date_1)

In [60]:
s1.day.unique()

array(['2020-01-07', '2020-01-08'], dtype=object)

In [61]:
s1

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,modificationtime,ph,entrydate_x,new_ph,entrydate_y,rbs,skintemp,centraltemp,temp,day
0,0,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2020-01-07
1,1,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2020-01-07
2,2,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2020-01-07
3,3,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2020-01-07
4,4,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2020-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,1520,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaT,NaN,NaT,NaN,2020-01-08 14:00:11.064,NaN,None,36.2,36.2,2020-01-08
1521,1521,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaT,NaN,NaT,NaN,2020-01-08 14:00:11.064,NaN,None,36.2,36.2,2020-01-08
1522,1522,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaT,NaN,NaT,NaN,2020-01-08 14:00:11.064,NaN,None,36.2,36.2,2020-01-08
1523,1523,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaT,NaN,NaT,NaN,2020-01-08 14:00:11.064,NaN,None,36.2,36.2,2020-01-08


In [62]:
s2 = pd.merge(s1,bv,on = ['uhid','day'], how='left')

In [63]:
len(s2.uhid.unique())

1

In [64]:
#Meds



In [65]:
cur7= con.cursor()
cur7.execute("SELECT t1.uhid,t1.startdate,t1.medicineorderdate,t1.medicinename,t1.medicationtype,t2.typevalue FROM apollo.baby_prescription AS t1 LEFT JOIN apollo.ref_medtype AS t2 ON t1.medicationtype = t2.typeid WHERE (t1.medicationtype = 'TYPE0001' OR t1.medicationtype = 'TYPE0009' OR t1.medicationtype = 'TYPE0004') and (t1.uhid ='"+uhidPatient+"')")
cols7 = list(map(lambda x: x[0], cur7.description))
meds = pd.DataFrame(cur7.fetchall(),columns=cols7)

In [66]:
meds

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue


In [67]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])


In [68]:
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)


In [69]:
meds_dummies

,uhid,startdate,medicineorderdate,medicinename,medicationtype,hour


In [70]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)
s2.drop(['hour_x','hour_y'],axis=1,inplace=True)

In [71]:
s2['hour'] = s2.hour_series.apply(split_hour)

In [72]:
meds_dummies

,uhid,startdate,medicineorderdate,medicinename,medicationtype,hour


In [73]:
s2

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,rbs,skintemp,centraltemp,temp,day,visitdate,visittime,currentdateweight,currentdateheight,hour
0,0,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
1,1,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
2,2,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
3,3,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
4,4,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,NaN,NaN,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,1520,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,36.2,36.2,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14
1521,1521,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,36.2,36.2,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14
1522,1522,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,36.2,36.2,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14
1523,1523,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,NaN,None,36.2,36.2,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14


In [74]:
s3 = pd.merge(s2,meds_dummies,on = ['uhid','hour'], how='left')

In [75]:
s3['day'] = s3['hour_series'].apply(split_date_1)

In [76]:
s3

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,day,visitdate,visittime,currentdateweight,currentdateheight,hour,startdate,medicineorderdate,medicinename,medicationtype
0,0,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13,NaN,NaN,NaN,NaN
1,1,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13,NaN,NaN,NaN,NaN
2,2,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13,NaN,NaN,NaN,NaN
3,3,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13,NaN,NaN,NaN,NaN
4,4,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-07,2020-01-07,12:29:00,2846.0,None,2020-01-07 13,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,1520,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14,NaN,NaN,NaN,NaN
1521,1521,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14,NaN,NaN,NaN,NaN
1522,1522,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14,NaN,NaN,NaN,NaN
1523,1523,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,2020-01-08,NaN,NaN,NaN,NaN,2020-01-08 14,NaN,NaN,NaN,NaN


In [77]:
#PN
cur8= con.cursor()
cur8.execute("SELECT t1.uhid,t1.entrydatetime,t1.totalparenteralvolume,t1.total_intake FROM apollo.babyfeed_detail AS t1 where t1.uhid = '"+uhidPatient+"'")
cols8 = list(map(lambda x: x[0], cur8.description))
pn = pd.DataFrame(cur8.fetchall(),columns=cols8)

In [78]:
pn['hour'] = pn.entrydatetime.apply(split_hour)
pn['total_intake'] = pn['total_intake'].astype(float)
pn['tpn-tfl'] = pn['totalparenteralvolume']/pn['total_intake']

In [79]:
s4 = pd.merge(s3,pn,on = ['uhid','hour'], how='left')

In [80]:
s4

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,currentdateheight,hour,startdate,medicineorderdate,medicinename,medicationtype,entrydatetime,totalparenteralvolume,total_intake,tpn-tfl
0,0,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,None,2020-01-07 13,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,1,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,None,2020-01-07 13,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,2,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,None,2020-01-07 13,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,3,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,None,2020-01-07 13,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,4,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,None,2020-01-07 13,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,1520,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,None,2020-01-08 14,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1521,1521,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,None,2020-01-08 14,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1522,1522,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,None,2020-01-08 14,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1523,1523,2020-01-07 12:29:00,RNEH.0000013713,2020-01-08 14:56:00,2020-01-07 13:31:00,Male,2846.0,NaN,36.0,In Born,...,None,2020-01-08 14,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [81]:
len(s4.uhid.unique())

1

In [82]:
s4.drop_duplicates(subset=['uhid','hour_series'],inplace=True)

In [83]:
q = pd.DataFrame()

In [84]:
for i in s4.uhid.unique():
    x = s4[s4['uhid']==i]
    x['currentdateheight'].fillna(method='ffill',inplace=True)
    x['currentdateweight'].fillna(method='ffill',inplace=True)
    q = q.append(x,ignore_index=True)

In [85]:
def abd(x):
    
    try:   
        
        if x>200:
            return x/10
        else:
            return x
    except:
        pass
    

In [86]:
q['abdomen_girth'] = pd.to_numeric(q['abdomen_girth'], errors='coerce')
q['abdomen_girth'] = q['abdomen_girth'].apply(abd)

In [87]:
dt = pd.DataFrame()
for i in q.uhid.unique():
    x = q[q['uhid']==i]
    #for j in y['day'].unique():
    #x = y[y['day']==j]
    x['abdomen_girth'].fillna(method='bfill',inplace=True)
    x['currentdateweight'].fillna(method='bfill',inplace=True)
    x['currentdateheight'].fillna(method='bfill',inplace=True)
    x['totalparenteralvolume'].fillna(method='ffill',inplace=True)
    x['total_intake'].fillna(method='ffill',inplace=True)
    x['tpn-tfl'].fillna(method='ffill',inplace=True)
    dt = dt.append(x,ignore_index=True)

In [88]:
dt.urine_per_hour.max()

inf

In [89]:
dt.urine_per_hour.unique()

array([1.21971786,        inf, 6.26021502])

In [90]:
a = dt.replace([np.inf, -np.inf], np.nan)

In [91]:
def abd_2(x):
    if not(x is None) and x < 0:
        return ((-1)*x)/10
    elif not(x is None) and x > 200:
        return x/10
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_2)

def abd_3(x):
    if not(x is None) and x < 15:
        return np.nan
    elif not(x is None) and x > 50:
        return np.nan
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_3)

def upm(x):
    if not(x is None) and x > 100:
        return np.nan
    else:
        return x
    
a['urine_per_hour'] = a['urine_per_hour'].apply(upm)

def temp_f_c(x):
    if not(x is None) and x > 90:
        return (x - 32)*(5/9)
    else:
        return x
    
a['temp'] = a['temp'].apply(temp_f_c)

def weight_correct(x):
    if not(x is None) and x<10:
        return x*100
    elif x>5000:
        return x/10
    else:
        return x
    
def weight_correct_2(x):
    if not(x is None) and x<300:
        return x*10
    elif x>5000:
        return x/10
    else:
        return x
    
a.currentdate = a.currentdateweight.apply(weight_correct)
a.currentdate = a.currentdateweight.apply(weight_correct_2)

def height_correct(x):
    if not(x is None) and x < 10:
        return np.nan
    else:
        return x
    
def rbs_correct(x):
    if not(x is None) and x > 700:
        return np.nan
    else:
        return x
    
a['currentdateheight'] = a['currentdateheight'].apply(height_correct)

a['rbs'] = a['rbs'].apply(rbs_correct)



/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


In [92]:
len(a.uhid.unique())

1

In [93]:
data = a.copy()

In [94]:
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass

In [95]:
data['day'] = data['hour_series'].apply(split_date_1)

In [96]:
def to_date(x):
    return pd.to_datetime(x)

In [97]:
data['hour_series'] = data['hour_series'].apply(to_date)

In [98]:
len(data.uhid.unique())

1

In [99]:
data.to_csv(caseType+seperator+uhidPatient+seperator+"intermittent_checkpoint_2.csv")

In [100]:
df = pd.DataFrame()
for i in data.uhid.unique():
    x = data[data['uhid']==i]
    x.rbs.fillna(method='ffill',inplace=True)
    x.currentdateweight.fillna(method='ffill',inplace=True)
    x.currentdateheight.fillna(method='ffill',inplace=True)
    n = math.ceil(len(x)/24)+1
    start_date = pd.to_datetime(x['day'].iloc[0]+" " + "08:00:00") - timedelta(hours=24)
    for i in range(int(n)):
        
       
        y = x[(x['hour_series']>=start_date + timedelta(hours=24*i)) & (x['hour_series']<=start_date + timedelta(hours=24*(i+1)))]
        y['stool_day_total'] = y['stool_passed'].sum()
        df =df.append(y,ignore_index=True)


/Users/harpreet/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [101]:
df.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_3.csv')

In [102]:
test = df.copy()

In [103]:
dq = pd.DataFrame()

In [104]:
for i in test.uhid.unique():
    x = test[test['uhid']==i]
    x.abdomen_girth.fillna(method='ffill',inplace=True)
    dq = dq.append(x, ignore_index=True)

In [105]:
final = pd.DataFrame()

In [106]:
for i in dq.uhid.unique():
    x = dq[dq['uhid']==i]
    x['abd_difference_y'] = x['abdomen_girth'] - x['abdomen_girth'].shift(periods=1)
    final = final.append(x,ignore_index=True)

In [107]:
final.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_4.csv')

In [108]:
cur1 = con.cursor()
cur1.execute("SELECT t1.uhid,t1.starttime,t1.pulserate, t1.ecg_resprate, t1.spo2, t1.heartrate, t2.dischargestatus,t1.mean_bp,t1.sys_bp,t1.dia_bp  FROM apollo.device_monitor_detail_dump AS t1 LEFT JOIN apollo.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE (t2.dischargestatus = 'Discharge' AND t2.dateofadmission > '2018-07-01' and t2.uhid = '"+ uhidPatient+"') OR (t2.dischargestatus = '"+caseType+"' AND t2.dateofadmission > '2018-07-01' and t2.uhid = '"+ uhidPatient+"');")
cols1 = list(map(lambda x: x[0], cur1.description))
ds = pd.DataFrame(cur1.fetchall(),columns=cols1)
ds.sort_values('starttime',inplace=True)

In [109]:
ds

,uhid,starttime,pulserate,ecg_resprate,spo2,heartrate,dischargestatus,mean_bp,sys_bp,dia_bp
0,RNEH.0000013713,2020-01-08 11:16:36+08:00,160,None,52,None,Death,52,71,43
1,RNEH.0000013713,2020-01-08 11:17:36+08:00,158,None,44,None,Death,52,71,43
6,RNEH.0000013713,2020-01-08 11:18:36+08:00,155,None,52,None,Death,52,71,43
5,RNEH.0000013713,2020-01-08 11:19:36+08:00,157,None,56,None,Death,52,71,43
2,RNEH.0000013713,2020-01-08 11:20:36+08:00,156,None,47,None,Death,52,71,43
...,...,...,...,...,...,...,...,...,...,...
579,RNEH.0000013713,2020-01-08 21:15:34+08:00,45,None,15,None,Death,None,None,None
586,RNEH.0000013713,2020-01-08 21:21:34+08:00,46,None,43,None,Death,None,None,None
587,RNEH.0000013713,2020-01-08 21:23:34+08:00,84,None,40,None,Death,None,None,None
585,RNEH.0000013713,2020-01-08 21:24:34+08:00,131,None,70,None,Death,None,None,None


In [110]:
test = ds.drop_duplicates(subset=['uhid','starttime'],keep='first')

In [111]:
cur_vent = con.cursor()
cur_vent.execute("SELECT t1.uhid,t1.start_time,t1.creationtime,t1.peep, t1.pip,t1.map ,t1.tidalvol, t1.minvol,t1.ti,t1.fio2 FROM apollo.device_ventilator_detail_dump AS t1 LEFT JOIN apollo.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE (t2.dischargestatus = 'Discharge' OR t2.dischargestatus = '"+caseType+"') and t2.uhid = '"+uhidPatient+"';")
cols_vent = list(map(lambda x: x[0], cur_vent.description))
ventilator_cont = pd.DataFrame(cur_vent.fetchall(),columns=cols_vent)

test_vent = ventilator_cont.drop_duplicates(subset=['uhid','start_time'],keep='first')

def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

test['date'] = test.starttime.apply(split_date)
test_vent['date'] = test_vent.start_time.apply(split_date)

In [112]:
cont_data = pd.merge(test,test_vent,on=['uhid','date'],how='left',copy=False)
test_cont = cont_data.drop_duplicates(subset=['uhid','starttime','heartrate'],keep='first')


In [113]:
test_cont['hour_series'] = test_cont['date'].apply(split_hour)

In [114]:
def to_str(x):
    return str(x)

In [115]:
test_cont['ref_hour'] = test_cont['hour_series'].apply(to_str)
final['ref_hour'] = final['hour_series'].apply(to_str)

In [116]:
def con_time(x):
    return str(x).split("+")[0].split(":")[0] + ":" +str(x).split("+")[0].split(":")[1]

In [117]:
test_cont['cont_time'] = test_cont.starttime.apply(con_time)

In [118]:
def con_time_2(x):
    return str(x).split(":")[0] +":"+str(x).split(":")[1]

In [119]:
final['cont_time'] = final.ref_hour.apply(con_time_2)

In [120]:
qw = pd.merge(final,test_cont, on=['uhid','cont_time'],how='left')

In [121]:
#final[final['cont_time']=='2019-09-30 11:18']

In [122]:
#qw.dropna(subset=['series'],inplace=True)

In [123]:
qw.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_5.csv')

In [124]:
qw.count()

Unnamed: 0        1526
actual_DOB        1526
uhid              1526
dischargeddate    1526
actual_DOA        1526
                  ... 
minvol               0
ti                   0
fio2                 0
hour_series_y      214
ref_hour_y         214
Length: 100, dtype: int64